In [1]:
import sys
sys.path.append('../src')

In [2]:
from gee_tools import Catalog, ZonalStats
import ee
import pandas as pd
import os
import glob

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
ee.Initialize()

In [5]:
cat = Catalog()

## NDVI

In [6]:
results = cat.search_tags("ndvi")
results = results.search_title("landsat")
results = results.search_title("8-day")

In [7]:
results.datasets

,id,provider,title,start_date,end_date,startyear,endyear,type,tags,asset_url,thumbnail_url
162,LANDSAT/LC08/C01/T1_8DAY_NDVI,Google,Landsat 8 Collection 1 Tier 1 8-Day NDVI Composite,2013-04-11,2021-03-09,2013,2021,image_collection,[ndvi],https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_8DAY_NDVI,https://mw1.google.com/ges/dd/images/NDVI_sample.png
195,LANDSAT/LE07/C01/T1_8DAY_NDVI,Google,Landsat 7 Collection 1 Tier 1 8-Day NDVI Composite,1999-01-01,2021-03-09,1999,2021,image_collection,[ndvi],https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_8DAY_NDVI,https://mw1.google.com/ges/dd/images/NDVI_sample.png
244,LANDSAT/LT04/C01/T1_8DAY_NDVI,Google,Landsat 4 TM Collection 1 Tier 1 8-Day NDVI Composite,1982-08-22,1993-12-14,1982,1993,image_collection,[ndvi],https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT04_C01_T1_8DAY_NDVI,https://mw1.google.com/ges/dd/images/NDVI_sample.png
275,LANDSAT/LT05/C01/T1_8DAY_NDVI,Google,Landsat 5 TM Collection 1 Tier 1 8-Day NDVI Composite,1984-01-01,2012-05-05,1984,2012,image_collection,[ndvi],https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C01_T1_8DAY_NDVI,https://mw1.google.com/ges/dd/images/NDVI_sample.png


In [97]:
AOIs = ee.FeatureCollection('users/afche18/Admin0_Polys_1000m')

In [13]:
temp_stats = ["mean", "max", "min", "stddev"]

In [10]:
ndvi_zs = {}

In [12]:
for landsat_collection in results.datasets.id:
    print(f"Processing {landsat_collection}")
    lc_id = landsat_collection[landsat_collection.find("/")+1:landsat_collection.find("/")+5]
    
    collection_results = []
    for stat in temp_stats:
        
        zs = ZonalStats(
            collection_id=landsat_collection, 
            target_features=AOIs,
            statistic_type="all", 
            output_name=f"{lc_id}_ndvi_{stat}",
            scale=1000,
            min_threshold=0.1,
            water_mask=True,
            freq='annual',
            temporal_stat=stat,
            tile_scale = 4
        )
        zs.runZonalStats()
        collection_results.append(zs)
    
    ndvi_zs[lc_id] = collection_results

Processing LANDSAT/LC08/C01/T1_8DAY_NDVI
Processing LANDSAT/LE07/C01/T1_8DAY_NDVI
Processing LANDSAT/LT04/C01/T1_8DAY_NDVI
Processing LANDSAT/LT05/C01/T1_8DAY_NDVI


In [256]:
for lc_id, zs_list in ndvi_zs.items():
    print(lc_id)
    for zs in zs_list:
        zs.reportRunTime()
#         print("\t"+zs.task.status()['state'])

LC08
Completed
Runtime: 0 minutes and 5 seconds
Completed
Runtime: 0 minutes and 6 seconds
Completed
Runtime: 0 minutes and 7 seconds
Completed
Runtime: 21 minutes and 52 seconds
LE07
Completed
Runtime: 34 minutes and 13 seconds
Completed
Runtime: 31 minutes and 23 seconds
Completed
Runtime: 33 minutes and 0 seconds
Completed
Runtime: 33 minutes and 22 seconds
LT04
Completed
Runtime: 4 minutes and 39 seconds
Completed
Runtime: 7 minutes and 29 seconds
Completed
Runtime: 4 minutes and 18 seconds
Completed
Runtime: 4 minutes and 16 seconds
LT05
Completed
Runtime: 28 minutes and 8 seconds
Completed
Runtime: 27 minutes and 20 seconds
Completed
Runtime: 27 minutes and 1 seconds
Completed
Runtime: 26 minutes and 35 seconds


In [18]:
def process_file(file_path):
    
    df = pd.read_csv(file_path, index_col=0)
    df.reset_index(drop=True, inplace=True)
    df.drop(columns=".geo", inplace=True)
    df.loc[:, "n_null"] = df.apply(lambda x: x.isna().sum(), axis=1)
#     df = df.loc[df.n_null<df.n_null.max()]
    
    file = os.path.basename(file_path)
    lc_id, var, temp_stat = file.replace(".csv", "").split("_")
    var_name = var.upper()
    var = "_".join([var, temp_stat])
    stubs = ["_".join([var, a]) for a in temp_stats]
    
    def rename_func(col):
        if var_name in col:
            p = col.split("_")
            new_name = p[1]+"_"+p[2]+"_"+p[3]+"__"+p[0]
            return new_name
        else:
            return col
    
    df.rename(rename_func, axis=1, inplace=True)
    df.rename(str.lower, axis=1, inplace=True)
    
    df_re = pd.wide_to_long(
        df,
        stubnames = stubs,
        i = 'wb_adm0_na',
        j = 'year',
        sep = "__"
    )
    
    return df_re

In [92]:
# data_dir = os.path.join(os.path.expanduser("~"), "Downloads")
data_dir = os.path.join("X:/data/ee")
ndvi_dir = os.path.join(data_dir, "ndvi")

In [69]:
def process_collection(lc_id, files_dir):
    files = glob.glob(files_dir+f"/{lc_id}*")
    l_processed = [process_file(file) for file in files]
    l_all = pd.concat(l_processed, axis=1)
    l_all = l_all.loc[:, ~l_all.columns.duplicated()].copy()
    l_all.loc[:, "landsat_id"] = lc_id
    return l_all

In [70]:
lc_ids = ["LT05","LT04","LE07","LC08"]

In [72]:
res = [process_collection(lc_id, ndvi_dir) for lc_id in lc_ids]

In [80]:
ndvi = pd.concat(res, axis=0)

In [84]:
ndvi.head()

,,region,longx,wb_adm0_co,objectid,shape_leng,shape_area,lendingc,fid_100,n_null,un_m49,...,ndvi_mean_stddev,ndvi_min_mean,ndvi_min_max,ndvi_min_min,ndvi_min_stddev,ndvi_stddev_mean,ndvi_stddev_max,ndvi_stddev_min,ndvi_stddev_stddev,landsat_id
wb_adm0_na,year,,,,,,,,,,,,,,,,,,,,,
Palmyra Atoll (U.S.),1984,NaN,-162.433879,190,171,1.959129e+04,8.520267,NaN,17000,115,581,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LT05
Pitcairn Islands (U.K.),1984,NaN,-128.379565,197,177,5.426991e+04,48.846457,NaN,17600,115,612,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LT05
Romania,1984,NaN,24.988376,203,183,4.042455e+06,238118.020404,NaN,18200,3,642,...,0.069123,0.145123,0.436650,0.100008,0.045366,0.176018,0.493399,0.100000,0.035404,LT05
"Saint Helena, Ascension and Tristan da Cunha (U.K.)",1984,NaN,-10.349737,207,186,2.236599e+05,410.586260,NaN,18500,115,654,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LT05
Saint-Pierre-et-Miquelon (Fr.),1984,NaN,-56.327116,210,189,2.369267e+05,228.038687,NaN,18800,19,666,...,0.086466,0.221855,0.441662,0.102969,0.090933,0.149610,0.227456,0.100085,0.032024,LT05


In [90]:
ndvi.loc["Uganda"].head()

,region,longx,wb_adm0_co,objectid,shape_leng,shape_area,lendingc,fid_100,n_null,un_m49,...,ndvi_mean_stddev,ndvi_min_mean,ndvi_min_max,ndvi_min_min,ndvi_min_stddev,ndvi_stddev_mean,ndvi_stddev_max,ndvi_stddev_min,ndvi_stddev_stddev,landsat_id
year,,,,,,,,,,,,,,,,,,,,,
1984,Sub-Saharan Africa,32.391496,253,226,2.588540e+06,241495.159157,IDA,22500,20,800,...,0.116260,0.295688,0.746945,0.100001,0.159135,0.171805,0.518526,0.100004,0.042978,LT05
1985,Sub-Saharan Africa,32.391496,253,226,2.588540e+06,241495.159157,IDA,22500,20,800,...,0.132230,0.283866,0.707618,0.100004,0.135296,0.141957,0.553882,0.100008,0.037616,LT05
1986,Sub-Saharan Africa,32.391496,253,226,2.588540e+06,241495.159157,IDA,22500,20,800,...,0.102061,0.208122,0.630003,0.100001,0.088420,0.162530,0.360771,0.100000,0.037646,LT05
1987,Sub-Saharan Africa,32.391496,253,226,2.588540e+06,241495.159157,IDA,22500,20,800,...,0.103353,0.217468,0.736020,0.100001,0.134141,0.165408,0.420269,0.100000,0.036415,LT05
1988,Sub-Saharan Africa,32.391496,253,226,2.588540e+06,241495.159157,IDA,22500,20,800,...,0.063640,0.185641,0.645917,0.100038,0.094238,0.174365,0.347420,0.100068,0.032993,LT05


In [93]:
out_dir = os.path.join(data_dir, "output")
if not os.path.exists(out_dir):
    os.mkdir(out_dir)

In [94]:
ndvi.to_csv(os.path.join(out_dir, "ndvi.csv"))

## EVI

In [87]:
results = cat.search_tags("evi")
results = results.search_title("landsat")
results = results.search_title("8-day")

In [89]:
results.datasets

,id,provider,title,start_date,end_date,startyear,endyear,type,tags,asset_url,thumbnail_url
159,LANDSAT/LC08/C01/T1_8DAY_EVI,Google,Landsat 8 Collection 1 Tier 1 8-Day EVI Composite,2013-04-11,2021-03-09,2013,2021,image_collection,[evi],https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LC08_C01_T1_8DAY_EVI,https://mw1.google.com/ges/dd/images/EVI_sample.png
192,LANDSAT/LE07/C01/T1_8DAY_EVI,Google,Landsat 7 Collection 1 Tier 1 8-Day EVI Composite,1999-01-01,2021-03-09,1999,2021,image_collection,[evi],https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LE07_C01_T1_8DAY_EVI,https://mw1.google.com/ges/dd/images/EVI_sample.png
241,LANDSAT/LT04/C01/T1_8DAY_EVI,Google,Landsat 4 TM Collection 1 Tier 1 8-Day EVI Composite,1982-08-22,1993-12-14,1982,1993,image_collection,[evi],https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT04_C01_T1_8DAY_EVI,https://mw1.google.com/ges/dd/images/EVI_sample.png
272,LANDSAT/LT05/C01/T1_8DAY_EVI,Google,Landsat 5 TM Collection 1 Tier 1 8-Day EVI Composite,1984-01-01,2012-05-05,1984,2012,image_collection,[evi],https://developers.google.com/earth-engine/datasets/catalog/LANDSAT_LT05_C01_T1_8DAY_EVI,https://mw1.google.com/ges/dd/images/EVI_sample.png


In [95]:
evi_zs = {}

In [98]:
for landsat_collection in results.datasets.id:
    print(f"Processing {landsat_collection}")
    lc_id = landsat_collection[landsat_collection.find("/")+1:landsat_collection.find("/")+5]
    
    collection_results = []
    for stat in temp_stats:
        
        zs = ZonalStats(
            collection_id=landsat_collection, 
            target_features=AOIs,
            statistic_type="all", 
            output_name=f"{lc_id}_evi_{stat}",
            scale=1000,
            min_threshold=0.1,
            water_mask=True,
            freq='annual',
            temporal_stat=stat,
            tile_scale = 4
        )
        zs.runZonalStats()
        collection_results.append(zs)
    
    evi_zs[lc_id] = collection_results

Processing LANDSAT/LC08/C01/T1_8DAY_EVI
Processing LANDSAT/LE07/C01/T1_8DAY_EVI
Processing LANDSAT/LT04/C01/T1_8DAY_EVI
Processing LANDSAT/LT05/C01/T1_8DAY_EVI


In [109]:
for lc_id, zs_list in evi_zs.items():
    print(lc_id)
    for zs in zs_list:
        zs.reportRunTime()
#         print("\t"+zs.task.status()['state'])

LC08
Failed!
User memory limit exceeded.
Still running
Runtime: 6 minutes and 24 seconds
Still running
Runtime: 3 minutes and 0 seconds
Status is Ready, hasn't started
LE07
Status is Ready, hasn't started
Status is Ready, hasn't started
Status is Ready, hasn't started
Status is Ready, hasn't started
LT04
Status is Ready, hasn't started
Status is Ready, hasn't started
Status is Ready, hasn't started
Status is Ready, hasn't started
LT05
Status is Ready, hasn't started
Status is Ready, hasn't started
Status is Ready, hasn't started
Status is Ready, hasn't started


In [108]:
evi_zs['LC08'][0].task.status()

{'state': 'FAILED',
 'description': 'Zonal statistics all of mean T1_8DAY_EVI',
 'creation_timestamp_ms': 1617371231065,
 'update_timestamp_ms': 1617371425299,
 'start_timestamp_ms': 1617371280453,
 'task_type': 'EXPORT_FEATURES',
 'attempt': 1,
 'error_message': 'User memory limit exceeded.',
 'id': 'QDEE2A4IDHHCWEW7AKEEJCF2',
 'name': 'projects/earthengine-legacy/operations/QDEE2A4IDHHCWEW7AKEEJCF2'}

In [72]:
# out_file = ndvi_zs['LC08'][0].output_name+".csv"

In [238]:
# cols[["mean_mean" in name for name in cols]]

In [239]:
# pd.melt(
#     test,
#     id_vars=['WB_ADM0_NA', 'WB_ADM0_CO'],
#     value_vars = cols[["mean_mean" in name for name in cols]],
#     value_name
# )

In [240]:
# pd.melt?